In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.linalg as la
np.set_printoptions(precision=2, formatter={'float':lambda x: f"{x:5.02f}"})

In [2]:
import numpy as np
def createBox():
    pts = [
        [0,0,0],
        [0,0,1],
        [0,1,0],
        [1,0,0],
        [1,1,0],
        [0,1,1],
        [1,0,1],
        [1,1,1],
        ]
    return np.array(pts).astype("float32")

box = createBox()
box = box + [50,50,50]
box_native = box
box_native

array([[50.00, 50.00, 50.00],
       [50.00, 50.00, 51.00],
       [50.00, 51.00, 50.00],
       [51.00, 50.00, 50.00],
       [51.00, 51.00, 50.00],
       [50.00, 51.00, 51.00],
       [51.00, 50.00, 51.00],
       [51.00, 51.00, 51.00]])

In [3]:
# basic pinhole camera
K = [
    [100, 0, 50],
    [0, 100, 50],
    [0,0,1]
    ]
K_native = np.array(K).astype("float32")

E = [
    [1,0,0,1],
    [0,1,0,1],
    [0,0,1,1],
    [0,0,0,1]
]
E = np.array(E)

E_native = la.inv(E)[:3,:]

P_native = np.matmul(K_native,E_native)
P_native

array([[100.00,  0.00, 50.00, -150.00],
       [ 0.00, 100.00, 50.00, -150.00],
       [ 0.00,  0.00,  1.00, -1.00]])

In [4]:
# view box

box_hat = np.hstack((box_native,np.ones((box.shape[0],1))))

box_x_hat = np.matmul(P_native,box_hat.T).T
box_x = np.array([box_x_hat[i,:] / box_x_hat[i,-1] for i in range(box_x_hat.shape[0])])[:,:-1]
box_native_x_native = box_x # box_native observed by native camera
box_native_x_native

array([[150.00, 150.00],
       [148.00, 148.00],
       [150.00, 152.04],
       [152.04, 150.00],
       [152.04, 152.04],
       [148.00, 150.00],
       [150.00, 148.00],
       [150.00, 150.00]])

#### calibration should result in same Camera as the camera that observed the points
Some rows of rotation matrix may have wrong sign, however K changes as well to compensate
Fix K to get correct one

In [5]:
from src.pycv.perspective import calibrate_dlt, decompose_perspective_projection_matrix

P_native_dlt = calibrate_dlt(box_native_x_native, box_native)

K_native_dlt, R_native_dlt, T_native_dlt = decompose_perspective_projection_matrix(P_native_dlt)
print(f"K\n{K_native_dlt}\nR\n{R_native_dlt}\nT\n{T_native_dlt}")

K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
T
[ 1.00  1.00  1.00]


#### calibrate a second camera in another coordinate system _world_
We assume that caneras in both coordinate systems are the same camera, thus the projections of the box in both coordinate systems are the same

In [6]:
box2 = createBox()
box2 = box2 + [5,5,5]
box_world = box2
box_world

array([[ 5.00,  5.00,  5.00],
       [ 5.00,  5.00,  6.00],
       [ 5.00,  6.00,  5.00],
       [ 6.00,  5.00,  5.00],
       [ 6.00,  6.00,  5.00],
       [ 5.00,  6.00,  6.00],
       [ 6.00,  5.00,  6.00],
       [ 6.00,  6.00,  6.00]])

In [7]:
# view box2, box2_x should be identical to box_x as they are the same camera

box2_x = box_x.copy()
box_world_x_world = box2_x
box_world_x_world

array([[150.00, 150.00],
       [148.00, 148.00],
       [150.00, 152.04],
       [152.04, 150.00],
       [152.04, 152.04],
       [148.00, 150.00],
       [150.00, 148.00],
       [150.00, 150.00]])

we can now determine the camera that observes the box in world coordinates by calibration

In [8]:
# calibrate camera using world to image correspondences
P_world = calibrate_dlt(box_world_x_world, box_world) # same position, projects to same

K_world, R_world, T_world = decompose_perspective_projection_matrix(P_world)
print(f"K\n{K_world}\nR\n{R_world}\nT\n{T_world}")

K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
T
[-44.00 -44.00 -44.00]


both cameras have no reprojection error when observing the box in their respective coordinate system

In [9]:
from src.pycv.perspective import reprojection_error

repr_err_native_native = reprojection_error(P_native, box_native_x_native, box_native)
repr_err_world_world = reprojection_error(P_world, box_world_x_world, box_world)

print(f"native reprojection error \n{repr_err_native_native} \n\nworld reprojection error \n{repr_err_world_world}")


native reprojection error 
[ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00] 

world reprojection error 
[ 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00]


#### compute transformation that transforms _native_ points to _world_ points

##### By using ground truth points
Estimate transform using ground truth we know because of artificial setup

(Points are the boxes defined in respective coordinate space) 

In [10]:
from src.pycv.perspective import horn_affine_transformation

# estimate transformation using horns method
A_boxes = horn_affine_transformation(box,box2)
A = A_boxes
A = np.vstack((A,[0,0,0,1]))
P_world_A = np.matmul(P_native,la.inv(A))

#K_world_A, R_world_A, T_world_A = decompose_perspective_projection_matrix(P_world_A)
#print(f"K\n{K_world_A}\nR\n{R_world_A}\nT\n{T_world_A}")
#print(f"K\n{K_world}\nR\n{R_world}\nT\n{T_world}")
extrinsic_world_A = np.matmul(la.inv(K_native),P_world_A)
camera_pose_world_A = la.inv(np.vstack((extrinsic_world_A,[0,0,0,1])))
camera_pose_world_A

array([[ 1.00, -0.00,  0.00, -44.00],
       [ 0.00,  1.00,  0.00, -44.00],
       [ 0.00,  0.00,  1.00, -44.00],
       [ 0.00,  0.00,  0.00,  1.00]])

##### By transfering box through camera
Box_native is not available in ushichka dataset, so we generate box_native_transfered by identifying the cameras with each other and moving the world points through camera space as in dmcp algorithm.

In [11]:
# what we assume to be known
P_native
K_native
# known by annotation
box_native_x_native
box_world
None

In [12]:
# DMCP Step 1 calibrate extrinsic matrix of camera using annotations 

from src.pycv.perspective import solve_PnP

#extrinsic_matrix_world = np.matmul(la.inv(K_native), P_world_A) 
#extrinsic_matrix_world = np.hstack((R,tvec))
pose_matrix = solve_PnP(box_world,box_native_x_native,K_native)
pose_matrix_hat = np.vstack((pose_matrix, [0,0,0,1]))
extrinsic_matrix_world = la.inv(pose_matrix_hat)[:3,:]


###### important observation: when computing extrinsic matrix from P_world_A the algorithm works, however when computing from P_world_dlt _or_ P_world it fails 

In [13]:
# test difference between dlt and PnP
P_world_dlt = calibrate_dlt(box_native_x_native, box_world)

# show that P_world_dlt is correct by comparing position
K, R, T = decompose_perspective_projection_matrix(P_world)
print(f"{K}\n{R}\n{T}")

import cv2
retval, rvec, tvec = cv2.solvePnP(box_world.astype("float32"), box_native_x_native.astype("float32"), K_native.copy().astype("float32"),np.array([[0, 0, 0, 0]], dtype=np.float32))

R, _ = cv2.Rodrigues(rvec)
R
tvec
extrinsic_matrix_world = np.hstack((R,tvec))
extrinsic_matrix_world_hat = np.vstack((extrinsic_matrix_world, [0,0,0,1]))
camera_pose_matrix_world_hat = la.inv(extrinsic_matrix_world_hat) # correct pose
camera_pose_matrix_world_hat

cpm = solve_PnP(box_world, box_native_x_native, K_native)
cpm

[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
[-44.00 -44.00 -44.00]


array([[ 1.00, -0.00, -0.00, -44.00],
       [ 0.00,  1.00, -0.00, -44.00],
       [ 0.00,  0.00,  1.00, -44.00]])

In [14]:
# DMCP Step 2.1 transform world points into camera space

box_world_hat = np.hstack((box_world, np.ones((box_world.shape[0],1))))

box_world_camera = np.matmul(extrinsic_matrix_world, box_world_hat.T).T
box_world_camera

#extrinsic_matrix_hat = np.vstack((extrinsic_matrix,[0,0,0,1]))
#pose_matrix = la.inv(extrinsic_matrix_hat)
#pose_matrix

array([[49.00, 49.00, 49.00],
       [49.00, 49.00, 50.00],
       [49.00, 50.00, 49.00],
       [50.00, 49.00, 49.00],
       [50.00, 50.00, 49.00],
       [49.00, 50.00, 50.00],
       [50.00, 49.00, 50.00],
       [50.00, 50.00, 50.00]])

for reference we compare box in camera space of native camera

for dmcp to work, they need to be equal

In [15]:
_extrinsic_matrix = np.matmul(la.inv(K_native), P_native) 

_box_native_hat = np.hstack((box_native, np.ones((box_native.shape[0],1))))

_box_native_camera = np.matmul(_extrinsic_matrix, _box_native_hat.T).T
_box_native_camera

array([[49.00, 49.00, 49.00],
       [49.00, 49.00, 50.00],
       [49.00, 50.00, 49.00],
       [50.00, 49.00, 49.00],
       [50.00, 50.00, 49.00],
       [49.00, 50.00, 50.00],
       [50.00, 49.00, 50.00],
       [50.00, 50.00, 50.00]])

In [16]:
# DMCP Step 2.2 transform camera points into native space

extrinsic_matrix_native = np.matmul(la.inv(K_native), P_native) 
extrinsic_matrix_native_hat = np.vstack((extrinsic_matrix_native,[0,0,0,1]))
pose_matrix = la.inv(extrinsic_matrix_native_hat)
camera_pose_matrix_native = pose_matrix

#np.matmul(camera_pose_matrix_native, )
box_world_camera_hat = np.hstack((box_world_camera,np.ones((box_world_camera.shape[0],1))))
box_native_tf = np.matmul(camera_pose_matrix_native, box_world_camera_hat.T).T[:,:3]
box_native_tf

array([[50.00, 50.00, 50.00],
       [50.00, 50.00, 51.00],
       [50.00, 51.00, 50.00],
       [51.00, 50.00, 50.00],
       [51.00, 51.00, 50.00],
       [50.00, 51.00, 51.00],
       [51.00, 50.00, 51.00],
       [51.00, 51.00, 51.00]])

In [24]:
# DMCP Step 2.3

A_tf = horn_affine_transformation(box_native_tf, box_world)

A_tf_hat = np.vstack((A_tf,[0,0,0,1]))

print(f"\nA_tf\n{A_tf_hat}\n")

P_world_tf = np.matmul(P_native,la.inv(A_tf_hat))

K_world_tf, R_world_tf, T_world_tf = decompose_perspective_projection_matrix(P_world_tf)
print(f"K\n{K_world_tf}\nR\n{R_world_tf}\nT\n{T_world_tf}")

extrinsic_world_tf = np.matmul(la.inv(K_native),P_world_tf)
camera_pose_world_tf = la.inv(np.vstack((extrinsic_world_tf,[0,0,0,1])))
camera_pose_world_tf



A_tf
[[ 1.00 -0.00 -0.00 -45.00]
 [ 0.00  1.00 -0.00 -45.00]
 [ 0.00  0.00  1.00 -45.00]
 [ 0.00  0.00  0.00  1.00]]

K
[[-100.00  0.00 50.00]
 [-0.00 -100.00 50.00]
 [ 0.00  0.00  1.00]]
R
[[-1.00 -0.00 -0.00]
 [ 0.00 -1.00 -0.00]
 [-0.00 -0.00  1.00]]
T
[-44.00 -44.00 -44.00]


array([[ 1.00, -0.00,  0.00, -44.00],
       [ 0.00,  1.00,  0.00, -44.00],
       [ 0.00,  0.00,  1.00, -44.00],
       [ 0.00,  0.00,  0.00,  1.00]])

#### Validate DMCP algorithm implementation

In [23]:
from src.pycv.dmcp import dmcp

A_dmcp_hat = dmcp(K_native, P_native,box_native_x_native, box_world)
A_dmcp_hat

array([[ 1.00, -0.00, -0.00, -45.00],
       [ 0.00,  1.00, -0.00, -45.00],
       [ 0.00,  0.00,  1.00, -45.00],
       [ 0.00,  0.00,  0.00,  1.00]])